# Sentiment Analysis using the emotion dataset with 6 emotions
native pytorch

In [ ]:
from transformers import AutoTokenizer

chkpt="distilbert-base-uncased"
saveModeldir="saved-model"

tokenizer=AutoTokenizer.from_pretrained(chkpt)

## Get the emotion dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("emotion")

In [ ]:
classes={}
for i, mood in enumerate(ds['train'].features['label'].names):
    classes[i]=mood    
classes

## Let's see examples of each emotion

In [ ]:
mood=0 

for i in range (100):
    if ds['train']['label'][i] ==mood:
        s=ds['train']['text'][i]
        print(f'{classes[mood]}: {s}  ')
        mood+=1
        if mood==6:
            break

## Preprocessing the dataset to make it compatible with model's tokenizer

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(sample):
    return tokenizer(sample['text'], truncation=True)

tokenized_ds = ds.map(tokenize_fn, batched=True)

# remove unwanted columns that miodel is not expecting. renaming column so model can expect it

tokenized_ds=tokenized_ds.remove_columns('text')
tokenized_ds=tokenized_ds.rename_column('label', 'labels')
tokenized_ds
tokenized_ds['train'].column_names

## Using DataCollator with DataLoader

### DataCp;;ator provides dynamic padding. Wehn combined with dataloader, the amount of padding to use depends on the longest sentence in the current batch. This only works when using GPU, and pytorch

In [ ]:
from torch.utils.data import DataLoader

batchSize=128

collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dl = DataLoader(tokenized_ds["train"], shuffle=True, batch_size=batchSize, collate_fn=collator)
eval_dl = DataLoader(tokenized_ds["validation"], batch_size=batchSize, collate_fn=collator)
test_dl= DataLoader(tokenized_ds["test"], batch_size=batchSize, collate_fn=collator)

print(f'train dl has {len(train_dl) }  batches')
print(f'validation dl has {len(eval_dl) }  batches')
print(f'test dl has {len(test_dl) }  batches')

In [ ]:
batch=next(iter(train_dl))
{k: v.shape for k, v in batch.items()}

### Notice how the 2nd dimension of the batch changes between batches? that's dynamic padding thanks to the data collator.

## Importing the model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(chkpt, num_labels=len(classes), id2label=classes)
model.config.id2label

In [ ]:
#try with a batch first to test if model can process a batch
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

## Hyper-Parameters

In [ ]:
import math 

lr=1e-4
epochs=5
totalSteps=math.ceil(ds['train'].num_rows/batchSize)*epochs
print(f'training has {totalSteps} total steps')

## importing optimizer and learning rate scheduler

In [ ]:
from transformers import AdamW
from transformers import get_scheduler

opt=AdamW(model.parameters(), lr=lr)
scheduler = get_scheduler("cosine",optimizer=opt,num_warmup_steps=0,num_training_steps=totalSteps)

## Setup device to use GPU

In [ ]:
import torch 
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 
model.to(device=device)

## Setup training loop

In [ ]:
def getAccuracy(outs, batch):
    preds=torch.argmax(outs.logits, dim=1)
    y=torch.sum(preds==batch['labels']).item()
    return y

In [ ]:
from tqdm import tqdm 

def train():
    losses=0.
    accs=0
    model.train() 
    torch.set_grad_enabled(True)   

    for batch in tqdm(train_dl):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        losses+=(loss.item() *len(batch['labels']))
        accs+=getAccuracy(outputs, batch)
        loss.backward()        
        opt.step()
        scheduler.step()
        opt.zero_grad()
    return losses/ds['train'].num_rows , accs

In [ ]:
from tqdm import tqdm 

def evaluate(ldr):
    losses=0. 
    accs=0
    model.eval()
    torch.set_grad_enabled(False)
    
    for batch in tqdm(ldr):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss.item()
        losses+=(loss*len(batch['labels']))
        accs+=getAccuracy(outputs, batch)    
    return losses/ds['validation'].num_rows , accs

In [ ]:
import IPython.display as display 


def showResults(TLosses, TAccs, VLosses, VAccs):
    display.clear_output(wait=True)        
    print('epoch\tTLoss\t\t TAcc\t\t ValLoss\t ValAcc\n')
    i=1
    for tl, ta, vl, va in zip(TLosses, TAccs, VLosses, VAccs) :
        print(f'{i}\t {tl:.3f}\t\t {ta*100.:.3f}%\t {vl:.3f}\t\t {va*100.:.3f}% ')
        i+=1

In [ ]:
from tqdm.auto import tqdm

TrainLosses=[]
TrainAccs=[]
valLosses=[]
valAccs=[]

totalTrain=len(ds['train'])
totalVal=len(ds['validation'] )

for epoch in range(epochs):
    print(f'epoch: {epoch+1} / {epochs}  ')
    print('training...')
    loss, accs=train()
    TrainLosses.append(loss)
    acc=accs/totalTrain 
    TrainAccs.append(acc)
    
    #test validation set
    print('evaluating...')
    loss, accs=evaluate(eval_dl)
    acc=accs/totalVal
    valLosses.append(loss)
    valAccs.append(acc)    
    showResults(TrainLosses, TrainAccs, valLosses, valAccs)

## Let's plot the losses and accuracies

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(20,20) )
plt.subplot(1,2,1)
plt.title('Losses')
plt.plot(TrainLosses, label='train')
plt.plot(valLosses,label='val')
plt.legend()
plt.subplot(1,2,2)
plt.title('Accuracies')
plt.plot(TrainAccs, label='train')
plt.plot(valAccs, label='val')
plt.legend()
plt.show()

In [ ]:
loss, acc=evaluate(test_dl)
acc=acc/len(ds['test'])
print(f'test accuracy={acc*100.:.3f} %')

## Save the trained model

In [ ]:
model.save_pretrained(saveModeldir)

## Load the model for inference

In [ ]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

In [ ]:
from transformers import AutoModelForSequenceClassification 

model2= AutoModelForSequenceClassification.from_pretrained(saveModeldir)
#model2.config
model2.to(device)

In [ ]:
def getMood(samples):
    input=tokenizer(samples,padding=True, truncation=True, return_tensors="pt" ).to(device)
    out=model2(**input)
    res=torch.argmax(out.logits , dim=1)    
    return res.cpu()

In [ ]:
str1="i am really annoyed"
str2="my heart yearns for her return"
str3="my heart is torn apart"
str4="it's a great sunday today"
str5="i'm left speechless"
str6="i stepped into the unknown"

strs=[str1, str2, str3, str4, str5, str6]

out=getMood(strs)

for i, res in enumerate(out):
    print(f'{strs[i]}: {model2.config.id2label[res.item()]}')